In [6]:
# http://kuanbutts.com/2020/09/12/raptor-simple-example/

import partridge as ptg
import os

In [4]:
#!pip install partridge

In [7]:
os.chdir(r"C:\VY-Projects\Link21\notebooks")

In [8]:
# load a GTFS of AC Transit
path = 'BL_GTFS.zip'
_date, service_ids = ptg.read_busiest_date(path)
view = {'trips.txt': {'service_id': service_ids}}
feed = ptg.load_feed(path, view)

In [9]:
import geopandas as gpd
import pyproj
from shapely.geometry import Point

# convert all known stops in the schedule to shapes in a GeoDataFrame
gdf = gpd.GeoDataFrame(
    {"stop_id": feed.stops.stop_id.tolist()},
    geometry=[
        Point(lon, lat)
        for lat, lon in zip(
            feed.stops.stop_lat,
            feed.stops.stop_lon)])
gdf = gdf.set_index("stop_id")
gdf.crs = {'init': 'epsg:4326'}

# re-cast to meter-based projection to allow for distance calculations
aeqd = pyproj.Proj(
    proj='aeqd',
    ellps='WGS84',
    datum='WGS84',
    lat_0=gdf.iloc[0].geometry.centroid.y,
    lon_0=gdf.iloc[0].geometry.centroid.x).srs
gdf = gdf.to_crs(crs=aeqd)

C:\Users\vyadav\Anaconda3\envs\runway37\lib\site-packages\geopandas\_compat.py:112: UserWarning: The Shapely GEOS version (3.11.0-CAPI-1.17.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\vyadav\Anaconda3\envs\runway37\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [10]:
# let's use this example origin and destination
# to find the time it would take to go from one to another
from_stop_name = "Santa Clara Av & Mozart St"
to_stop_name = "10th Avenue SB"

# QA: we know the best way to connect these two is the 51A -> 1T
# if we depart at 8:30 AM, schedule should suggest:
#     take 51A 8:37 - 8:49
#     make walk connection
#     take 1T 8:56 - 9:03
# total travel time: 26 minutes

# look at all trips from that stop that are after the depart time
departure_secs = 8.5 * 60 * 60

# get all information, including the stop ids, for the start and end nodes
from_stop = feed.stops[feed.stops.stop_name == from_stop_name].head(1).squeeze()
to_stop = feed.stops[["10th Avenue" in f for f in feed.stops.stop_name]].head(1).squeeze()

# extract just the stop ids
from_stop_id = from_stop.stop_id
to_stop_id = to_stop.stop_id

In [13]:
from_stop_id

Series([], Name: stop_id, dtype: object)

In [14]:
from collections import deque
from typing import Dict, List, Tuple


def raptor_algorithm(
    network: Dict[int, List[Tuple[int, int]]],
    start: int,
    end: int,
    max_rounds: int = 100,
) -> List[int]:
    """
    RAPTOR algorithm for transit routing.

    Args:
        network (dict): A dictionary representing the transportation network.
            Keys are node IDs and values are lists of tuples representing edges.
            Each edge is a tuple of the form (destination node ID, travel time).
        start (int): The ID of the starting node.
        end (int): The ID of the ending node.
        max_rounds (int): The maximum number of rounds to run the algorithm.
            Defaults to 100.

    Returns:
        A list of node IDs representing the shortest path from the starting
        node to the ending node, or an empty list if no path is found.
    """
    # Initialize the state of the algorithm
    rounds = [set([start])]  # List of sets of reachable nodes for each round
    reached = [False] * len(network)  # List of flags indicating if a node has been reached
    reached[start] = True

    # Run the algorithm for the specified number of rounds
    for round_num in range(max_rounds):
        current_round = rounds[-1]
        next_round = set()

        # Iterate over all nodes in the current round
        for current_node in current_round:
            # Iterate over all edges from the current node
            for neighbor, travel_time in network[current_node]:
                # Check if the neighbor has already been reached in a previous round
                if not reached[neighbor]:
                    reached[neighbor] = True
                    next_round.add(neighbor)

        # Add the next round to the list of rounds
        rounds.append(next_round)

        # Check if the end node has been reached
        if reached[end]:
            break

    # If the end node was not reached, return an empty list
    if not reached[end]:
        return []

    # Build the shortest path from the starting node to the ending node
    path = [end]
    for round_num in range(len(rounds) - 2, -1, -1):
        current_round = rounds[round_num]
        for node in current_round:
            for neighbor, travel_time in network[node]:
                if neighbor == path[-1] and reached[node]:
                    path.append(node)
                    break

        # If the starting node is reached, stop building the path
        if path[-1] == start:
            break

    # Reverse the path and return it
    return path[::-1]

In [18]:
import csv
from collections import defaultdict
from typing import Dict, List, Tuple


def parse_gtfs_data(filename: str) -> Dict[int, List[Tuple[int, int]]]:
    """
    Parse a GTFS dataset and return a dictionary representing the transportation network.

    Args:
        filename (str): The name of the GTFS dataset file.

    Returns:
        A dictionary representing the transportation network.
        Keys are node IDs and values are lists of tuples representing edges.
        Each edge is a tuple of the form (destination node ID, travel time).
    """
    # Load the data from the CSV files
    stops = {}
    with open(filename + '/stops.txt', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            stops[row['stop_id']] = (float(row['stop_lat']), float(row['stop_lon']))

    routes = {}
    with open(filename + '/routes.txt', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            routes[row['route_id']] = row['route_short_name']

    trips = defaultdict(list)
    with open(filename + '/trips.txt', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            trips[row['route_id']].append(row['trip_id'])

    stop_times = {}
    with open(filename + '/stop_times.txt', newline='') as f:
        reader = csv.DictReader(f)
        for row in reader:
            stop_id = row['stop_id']
            trip_id = row['trip_id']
            arrival_time = row['arrival_time'] #int(row['arrival_time'])
            stop_times[trip_id, stop_id] = arrival_time

    # Build the transportation network
    network = defaultdict(list)
    for route_id, trip_ids in trips.items():
        for i in range(len(trip_ids)):
            trip_id1 = trip_ids[i]
            for j in range(i + 1, len(trip_ids)):
                trip_id2 = trip_ids[j]
                common_stops = set(stop_times.keys()) & {(trip_id1, stop_id) for stop_id in stops} & {(trip_id2, stop_id) for stop_id in stops}
                for stop_id in stops:
                    if (trip_id1, stop_id) in common_stops and (trip_id2, stop_id) in common_stops:
                        time1 = stop_times[trip_id1, stop_id]
                        time2 = stop_times[trip_id2, stop_id]
                        if time2 > time1:
                            network[stop_id].append((stop_id, time2 - time1))
                        else:
                            network[stop_id].append((stop_id, time2 + 24*60*60 - time1))

    # Convert the node IDs from strings to integers
    id_map = {stop_id: i for i, stop_id in enumerate(network.keys())}
    network = {id_map[stop_id]: [(id_map[dest], time) for dest, time in edges] for stop_id, edges in network.items()}

    return network

In [19]:
net = parse_gtfs_data(r"common_data\BL_GTFS")

In [20]:
net

{}